# Data Generation

The primary goal of this notebook is to generate additional features required for the entire ML projects.

In [2]:
import polars as pl
import pandas as pd
import duckdb as db
import numpy as np
import pandas as pd

In [16]:
df=pd.read_csv("../../data/raw/TRAIN.csv", parse_dates=['Date'])
df.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Orders,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [17]:
df["Day_of_Week_Name"] = df["Date"].dt.day_name()
df['Day'] = df['Date'].dt.day
df["Day_of_Week"] = df["Date"].dt.dayofweek
df["Month_Name"] = df["Date"].dt.month_name()
df["Month"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year
df["Quarter"] = df["Date"].dt.quarter
df["Week"] = df["Date"].dt.isocalendar().week
df["Week_of_Month"] = df["Date"].dt.day.apply(lambda x: (x-1) // 7 + 1)
df["Is_Weekend"] = df["Day_of_Week_Name"].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
df["Discount"] = df["Discount"].apply(lambda x: 1 if x == 'Yes' else 0)

In [18]:
df

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,Orders,Sales,Day_of_Week_Name,Day,Day_of_Week,Month_Name,Month,Year,Quarter,Week,Week_of_Month,Is_Weekend
0,T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84,Monday,1,0,January,1,2018,1,1,1,0
1,T1000002,253,S4,L2,R1,2018-01-01,1,1,60,51789.12,Monday,1,0,January,1,2018,1,1,1,0
2,T1000003,252,S3,L2,R1,2018-01-01,1,1,42,36868.20,Monday,1,0,January,1,2018,1,1,1,0
3,T1000004,251,S2,L3,R1,2018-01-01,1,1,23,19715.16,Monday,1,0,January,1,2018,1,1,1,0
4,T1000005,250,S2,L3,R4,2018-01-01,1,1,62,45614.52,Monday,1,0,January,1,2018,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,1,51,37272.00,Friday,31,4,May,5,2019,2,22,5,0
188336,T1188337,153,S4,L2,R1,2019-05-31,1,0,90,54572.64,Friday,31,4,May,5,2019,2,22,5,0
188337,T1188338,154,S1,L3,R2,2019-05-31,1,0,56,31624.56,Friday,31,4,May,5,2019,2,22,5,0
188338,T1188339,155,S3,L1,R2,2019-05-31,1,1,70,49162.41,Friday,31,4,May,5,2019,2,22,5,0


In [19]:
df.to_parquet('../../data/processed/train_enhanced.parquet', index=False)

In [11]:
db.sql("""
select distinct store_id
       , Region_Code
       , Location_Type
       , Store_Type
        from df
       order by 1,2,3,4
""")

┌──────────┬─────────────┬───────────────┬────────────┐
│ Store_id │ Region_Code │ Location_Type │ Store_Type │
│  int64   │   varchar   │    varchar    │  varchar   │
├──────────┼─────────────┼───────────────┼────────────┤
│        1 │ R1          │ L3            │ S1         │
│        2 │ R3          │ L1            │ S3         │
│        3 │ R1          │ L2            │ S4         │
│        4 │ R2          │ L1            │ S1         │
│        5 │ R3          │ L1            │ S1         │
│        6 │ R1          │ L1            │ S4         │
│        7 │ R3          │ L1            │ S3         │
│        8 │ R2          │ L2            │ S4         │
│        9 │ R1          │ L1            │ S1         │
│       10 │ R3          │ L2            │ S4         │
│        · │ ·           │ ·             │ ·          │
│        · │ ·           │ ·             │ ·          │
│        · │ ·           │ ·             │ ·          │
│      356 │ R1          │ L3            │ S3   

In [9]:
db.sql("""
select distinct
       Region_Code,
       count(distinct store_id) as store_count,
        from df
         group by 1
       
""")

┌─────────────┬─────────────┐
│ Region_Code │ store_count │
│   varchar   │    int64    │
├─────────────┼─────────────┤
│ R1          │         124 │
│ R2          │         105 │
│ R3          │          86 │
│ R4          │          50 │
└─────────────┴─────────────┘

In [30]:
holidays = db.sql("""
    select distinct Date, Holiday from df
 """).to_df()
holidays.head()

,Date,Holiday
0,2018-12-27,0
1,2018-12-29,0
2,2019-01-13,1
3,2019-01-27,0
4,2019-02-01,0


In [31]:
holidays.to_csv('../../data/processed/holidays.csv', index=False)

In [32]:
db.sql("""
    select distinct Store_Type,Date, Discount from df order by  Date
""")

┌────────────┬─────────────────────┬──────────┐
│ Store_Type │        Date         │ Discount │
│  varchar   │    timestamp_ns     │   int8   │
├────────────┼─────────────────────┼──────────┤
│ S1         │ 2018-01-01 00:00:00 │        1 │
│ S2         │ 2018-01-01 00:00:00 │        1 │
│ S3         │ 2018-01-01 00:00:00 │        1 │
│ S4         │ 2018-01-01 00:00:00 │        1 │
│ S4         │ 2018-01-02 00:00:00 │        1 │
│ S2         │ 2018-01-02 00:00:00 │        1 │
│ S3         │ 2018-01-02 00:00:00 │        1 │
│ S1         │ 2018-01-02 00:00:00 │        1 │
│ S2         │ 2018-01-03 00:00:00 │        1 │
│ S3         │ 2018-01-03 00:00:00 │        1 │
│ ·          │          ·          │        · │
│ ·          │          ·          │        · │
│ ·          │          ·          │        · │
│ S4         │ 2019-05-30 00:00:00 │        1 │
│ S3         │ 2019-05-30 00:00:00 │        1 │
│ S4         │ 2019-05-31 00:00:00 │        1 │
│ S1         │ 2019-05-31 00:00:00 │    

In [33]:
db.sql("""
    select distinct Store_id,Date, Discount from df where Store_id = 1 
""")

┌──────────┬─────────────────────┬──────────┐
│ Store_id │        Date         │ Discount │
│  int64   │    timestamp_ns     │   int8   │
├──────────┼─────────────────────┼──────────┤
│        1 │ 2018-01-05 00:00:00 │        1 │
│        1 │ 2018-01-06 00:00:00 │        0 │
│        1 │ 2018-01-11 00:00:00 │        0 │
│        1 │ 2018-01-17 00:00:00 │        1 │
│        1 │ 2018-02-01 00:00:00 │        0 │
│        1 │ 2018-02-08 00:00:00 │        1 │
│        1 │ 2018-02-16 00:00:00 │        0 │
│        1 │ 2018-02-21 00:00:00 │        1 │
│        1 │ 2018-03-06 00:00:00 │        0 │
│        1 │ 2018-03-15 00:00:00 │        1 │
│        · │          ·          │        · │
│        · │          ·          │        · │
│        · │          ·          │        · │
│        1 │ 2019-02-01 00:00:00 │        0 │
│        1 │ 2019-02-19 00:00:00 │        1 │
│        1 │ 2019-03-03 00:00:00 │        1 │
│        1 │ 2019-03-19 00:00:00 │        0 │
│        1 │ 2019-04-01 00:00:00 │

In [6]:
db.sql("""
with weekly_sales_table as (
    select Store_id,Week, Year, sum(Sales) as Weekly_Sales from df group by Store_id, Week, Year
)
select Store_id, Week, Year,
       Weekly_Sales,
       lag(Weekly_Sales, 1) over (partition by Store_id order by Year, Week) as Last_Week_Sales,
        (Weekly_Sales - lag(Weekly_Sales, 1) over (partition by Store_id order by Year, Week))/lag(Weekly_Sales, 1) over (partition by Store_id order by Year, Week) as Sales_Growth
       from weekly_sales_table
       order by Store_id, Year, Week
       """)

┌──────────┬────────┬───────┬────────────────────┬────────────────────┬──────────────────────┐
│ Store_id │  Week  │ Year  │    Weekly_Sales    │  Last_Week_Sales   │     Sales_Growth     │
│  int64   │ uint32 │ int32 │       double       │       double       │        double        │
├──────────┼────────┼───────┼────────────────────┼────────────────────┼──────────────────────┤
│        1 │      1 │  2018 │ 294654.83999999997 │               NULL │                 NULL │
│        1 │      2 │  2018 │           252701.4 │ 294654.83999999997 │ -0.14238164219532243 │
│        1 │      3 │  2018 │           254220.0 │           252701.4 │ 0.006009464134349892 │
│        1 │      4 │  2018 │          307899.63 │           254220.0 │   0.2111542364880812 │
│        1 │      5 │  2018 │          203948.94 │          307899.63 │  -0.3376122602031058 │
│        1 │      6 │  2018 │           224793.0 │          203948.94 │  0.10220234535173361 │
│        1 │      7 │  2018 │           192775.5 │

In [8]:
db.sql("""
with weekly_sales_table as (
    select Store_id,Week, Year, sum(Sales) as Weekly_Sales from df group by Store_id, Week, Year
),
growth_table as (
select Store_id, Week, Year,
       Weekly_Sales,
       lag(Weekly_Sales, 1) over (partition by Store_id order by Year, Week) as Last_Week_Sales,
        (Weekly_Sales - lag(Weekly_Sales, 1) over (partition by Store_id order by Year, Week))/lag(Weekly_Sales, 1) over (partition by Store_id order by Year, Week) as Sales_Growth
       from weekly_sales_table
       order by Store_id, Year, Week
)
select Store_id, avg(Sales_Growth) as Avg_Sales_Growth from growth_table group by Store_id order by 2 desc
       """)

┌──────────┬────────────────────────┐
│ Store_id │    Avg_Sales_Growth    │
│  int64   │         double         │
├──────────┼────────────────────────┤
│       67 │    0.06631537275528393 │
│       39 │   0.055243112317978106 │
│      197 │    0.04794921724996895 │
│      352 │    0.04681642541815307 │
│      178 │    0.04268551220372962 │
│       53 │   0.039082741694588106 │
│      286 │   0.037876340227856374 │
│      324 │    0.03779387692286832 │
│      271 │   0.037434803760623046 │
│      246 │    0.03729766544059077 │
│       ·  │             ·          │
│       ·  │             ·          │
│       ·  │             ·          │
│      279 │   0.000888684918665669 │
│       40 │   0.000854049233004496 │
│      220 │  0.0008528820491048724 │
│      157 │  0.0006181823430243793 │
│       69 │  0.0005740793325990584 │
│       86 │ -0.0002554436278111096 │
│        8 │ -0.0006793109381213872 │
│       88 │ -0.0020122708327546942 │
│       65 │ -0.0037838804245156803 │
│      159 │

In [15]:
db.sql("""
with Monthly_sales_table as (
    select Store_id,Month, Year, sum(Sales) as Monthly_Sales from df group by Store_id, Month, Year
),
growth_table as (
    select 
        Store_id, Month, Year,
        Monthly_Sales,
        lag(Monthly_Sales, 1) over (partition by Store_id order by Year, Month) as Last_Month_Sales
    from Monthly_sales_table
),
final_table as (
    select 
        Store_id, Month, Year,
        Monthly_Sales,
        Last_Month_Sales,
        (Monthly_Sales - Last_Month_Sales)/Last_Month_Sales as Sales_Growth
    from growth_table
        order by Store_id, Year, Month
)
select Store_id, avg(Sales_Growth) as Avg_Sales_Growth from final_table group by Store_id order by 2 desc
       """)

┌──────────┬───────────────────────┐
│ Store_id │   Avg_Sales_Growth    │
│  int64   │        double         │
├──────────┼───────────────────────┤
│        3 │   0.06400558593064012 │
│       82 │   0.05916034715638162 │
│      253 │    0.0580273162173353 │
│      356 │  0.049354995215687446 │
│      352 │  0.048702092181421075 │
│      268 │   0.04508706196132216 │
│       67 │   0.04419719465138049 │
│      335 │   0.04314888301058971 │
│      150 │    0.0428804288116823 │
│      280 │  0.042849005322063975 │
│        · │            ·          │
│        · │            ·          │
│        · │            ·          │
│       40 │ -0.003583669424599721 │
│      136 │ -0.003639835244974377 │
│       46 │ -0.004190984890044918 │
│      233 │ -0.008209307439991225 │
│      221 │ -0.008521134596028393 │
│      300 │ -0.008782261240966014 │
│      265 │ -0.009637302233560841 │
│       88 │ -0.012620862829776612 │
│      159 │  -0.01268893337189539 │
│       65 │  -0.02588240371310368 │
├

In [8]:
train_store_type_agg =db.sql("""
select 
        Date
        , Store_Type
        , cast(sum(Sales) as int) as Total_Sales
        , cast(avg(Sales) as int) as Avg_Sales
        , sum("Order") as Total_Orders
        , cast(avg("Order") as int) as Avg_Orders
        , count(distinct Store_id) as Num_Stores
        , max(Holiday) as Holiday
        , sum(Discount) as Total_Discounts
    from df
        group by 
       Date
       , Store_Type
    order by Date
""").pl()
train_store_type_agg.head()

Date,Store_Type,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
datetime[ns],str,i32,i32,"decimal[38,0]",i32,i64,i64,"decimal[38,0]"
2018-01-01 00:00:00,"""S4""",3728611,41895,4778,54,89,1,89
2018-01-01 00:00:00,"""S1""",7292028,42396,9410,55,172,1,172
2018-01-01 00:00:00,"""S3""",1944327,40507,2439,51,48,1,48
2018-01-01 00:00:00,"""S2""",2380519,42509,3039,54,56,1,56
2018-01-02 00:00:00,"""S4""",6515439,73207,8988,101,89,0,89


In [9]:
train_store_type_agg.write_parquet('../../data/processed/train_store_type_agg.parquet')

In [10]:
train_region_code_agg =db.sql("""
    select 
        Date
        , Region_Code
        , cast(sum(Sales) as int) as Total_Sales
        , cast(avg(Sales) as int) as Avg_Sales
        , sum("Order") as Total_Orders
        , cast(avg("Order") as int) as Avg_Orders
        , count(distinct Store_id) as Num_Stores
        , max(Holiday) as Holiday
        , sum(Discount) as Total_Discounts
    from df
        group by 
       Date
       , Region_Code
    order by Date
""").pl()
train_region_code_agg.head()

Date,Region_Code,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
datetime[ns],str,i32,i32,"decimal[38,0]",i32,i64,i64,"decimal[38,0]"
2018-01-01 00:00:00,"""R4""",2286812,45736,2914,58,50,1,50
2018-01-01 00:00:00,"""R2""",4436859,42256,5644,54,105,1,105
2018-01-01 00:00:00,"""R3""",3527439,41017,4599,53,86,1,86
2018-01-01 00:00:00,"""R1""",5094374,41084,6509,52,124,1,124
2018-01-02 00:00:00,"""R4""",2545119,50902,3057,61,50,0,50


In [11]:
train_region_code_agg.write_parquet('../../data/processed/train_region_code_agg.parquet')

In [12]:
train_location_type_agg =db.sql("""
    select 
        Date
        , Location_Type
        , cast(sum(Sales) as int) as Total_Sales
        , cast(avg(Sales) as int) as Avg_Sales
        , sum("Order") as Total_Orders
        , cast(avg("Order") as int) as Avg_Orders
        , count(distinct Store_id) as Num_Stores
        , max(Holiday) as Holiday
        , sum(Discount) as Total_Discounts
    from df
        group by 
       Date
       , Location_Type
    order by Date
""").pl()
train_location_type_agg.head()

Date,Location_Type,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
datetime[ns],str,i32,i32,"decimal[38,0]",i32,i64,i64,"decimal[38,0]"
2018-01-01 00:00:00,"""L3""",2460245,42418,3175,55,58,1,58
2018-01-01 00:00:00,"""L2""",4117915,43808,5295,56,94,1,94
2018-01-01 00:00:00,"""L1""",6754876,40939,8662,52,165,1,165
2018-01-01 00:00:00,"""L4""",841889,40090,1090,52,21,1,21
2018-01-01 00:00:00,"""L5""",1170560,43354,1444,53,27,1,27


In [13]:
train_location_type_agg.write_parquet('../../data/processed/train_location_type_agg.parquet')

In [14]:
train_agg =db.sql("""
    select 
        Date
        , cast(sum(Sales) as int) as Total_Sales
        , cast(avg(Sales) as int) as Avg_Sales
        , sum("Order") as Total_Orders
        , cast(avg("Order") as int) as Avg_Orders
        , count(distinct Store_id) as Num_Stores
        , max(Holiday) as Holiday
        , sum(Discount) as Total_Discounts
    from df
        group by 
       Date
    order by Date
""").pl()
train_location_type_agg

Date,Location_Type,Total_Sales,Avg_Sales,Total_Orders,Avg_Orders,Num_Stores,Holiday,Total_Discounts
datetime[ns],str,i32,i32,"decimal[38,0]",i32,i64,i64,"decimal[38,0]"
2018-01-01 00:00:00,"""L3""",2460245,42418,3175,55,58,1,58
2018-01-01 00:00:00,"""L2""",4117915,43808,5295,56,94,1,94
2018-01-01 00:00:00,"""L1""",6754876,40939,8662,52,165,1,165
2018-01-01 00:00:00,"""L4""",841889,40090,1090,52,21,1,21
2018-01-01 00:00:00,"""L5""",1170560,43354,1444,53,27,1,27
…,…,…,…,…,…,…,…,…
2019-05-31 00:00:00,"""L5""",642061,23780,1009,37,27,1,1
2019-05-31 00:00:00,"""L3""",1877677,32374,2883,50,58,1,5
2019-05-31 00:00:00,"""L1""",6926767,41980,10286,62,165,1,13


In [7]:
db.sql("""
    select 
        Date
        , cast(sum(Sales) as int) as Total_Sales
        , cast(avg(Sales) as int) as Avg_Sales
        , sum(Orders) as Total_Orders
        , cast(avg(Orders) as int) as Avg_Orders
        , max(Holiday) as Holiday
        , sum(Discount) as Total_Discounts
    from df
        group by 
       Date
    order by Date
""").pl().write_parquet('../../data/processed/train_agg.parquet')